## Random Forest Classifiers
<ins>R</ins>andom <ins>F</ins>orest <ins>C</ins>lassifiers use an ensemble method meaning models are created by combining the result of many individual decision tree models. Decision trees classify a set of features by combining (many) layers of sub-classifications. RFCs train small decision trees, that is ones with only a few layers, and use the most popular result of all the trees to classify the set of features. 

RFCs can produce powerful models and are particularly good at working with categoricals. However, disadvantage is they can train to favor some features more than others. Another drawback is that the model is not easily interpreted; that is, it will give clear results but the reason for each classification is hidden under the hood. In many situations RFC models perform as good or better than other types of models.

## Using RFC on the Credit Card Churn Data
RFC is a candidate for a good model for this data because of the size of the data, both numbers of samples and number of features. It is also expected to handle the many categorical features well. For predicting churn, it is not necessary to understand the models 'reasoning and so RFC's lack of interoperability should not be concerning. It is expected that RFC outperforms some of the other model types like logistic regression. 

The credit card data includes two broad types of features: demographic and usage. Demographic features should be considered easy to obtain while the latter require knowing a customer's usage history, which in some use cases does not exist. Because of this, a RFC model will be fitted on the demographic data, separately, to see if a functioning model can be made without usage history. The base model, trained on full data will serve as comparison to test the demographic models performance. 

It is also noted that, while this data set is medium sized (~25,000 samples) producing a model for a much larger set of data could be time consuming and resource demanding. Because of this, a RFC model will be fitted on a PCA dimensionally reduced set of data. 

This notebook uses helper functions from mechlearn.py to produce many of the scores and curves. More on the documentation for those functions can be found in `mechlearn.py`


In [ ]:
# import dependencies
import pandas as pd
from mechlearn import split_and_scale as ss
from mechlearn import auc_test as auct
from mechlearn import acc_test as acct
from mechlearn import feature_importances as fi
from matplotlib import pyplot as plt
import numpy as np

The 'Resources/X.csv' and 'Resources/y.csv' files were generated in the `Data_Exploration.ipynb` notebook and hold the cleaned data and target values for the entire dataset.

In [ ]:
# load data (X) and target (y) into dataframes
X = pd.read_csv('../Resources/X.csv')
y = pd.read_csv('../Resources/y.csv')

## Evaluate Model Using All Features
The first RFC model created, the base model, uses the full data set and is expected to perform the best since it uses all 37 input features. It scores 95% accuracy on the testing set of data. This is significantly higher than some of the other models trained like logistic regression and k-nearest neighbors, both scoring around 89% accuracy on testing dat. The base RFC model also scores a 99% AUC-ROC which means that the model can be tuned to favor low false positive rate or high true positive rate base of the use case. 

The model also gives some insight on how the demographics and pca models will perform by showing the feature importances. The graph of feature importances shows which features are used the most across the random forest. Usage features like `Total_Trans_Ct` and `Total_Trans_Amt` show the highest importance. Demographic features like `Gender`, `Education_Level` and `Marital_Status` show little importances to the base model. This could indicate the demographic model will have trouble fitting to just the demographic data.

In [ ]:
features = X.columns
print(f'Number of Features: {len(features)}')
print('----------------------------------------------------------------------------')
print(f'Feature names: {features}')

In [ ]:
print('Random forest classifiier on full data')
print(f'Accuracy: {acct(X, y, model="rfc")}')
print(f'Area under curve: {auct(X, y, model="rfc")}')

In [ ]:
plot = pd.read_csv('../Outputs/RandomForestClassifier_ROC.csv')
plt.plot(plot['fpr'], plot['tpr'])
plt.xlabel('False Postive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC for Random Forest Classifier Using All Features')
plt.show()

In [ ]:
feature_importances = fi(X, y, model='rfc')
ticks = np.arange(len(X.columns))
fig, ax = plt.subplots()
ax.barh(ticks, feature_importances)
ax.set_yticks(ticks, labels=X.columns)
fig.set_size_inches(15, 15)
ax.set_xlabel('Feature Importances')
ax.set_title('Feature Importances for Random Forest Classifier')
plt.show()
pd.DataFrame({'Features': features, 'Feature_Importances': feature_importances}).to_csv('../Outputs/RandomForestClassifier_FI.csv')

## Evaluate Model with Just Demographics Data
The demographic RFC model is created in the same way as the base model except it is only given demographic features data to train on. It scores an accuracy of 77% on the testing data. Not only is this a significantly lower accuracy than the logistic regression and knn models but it is also less than random chance. The initial data is 86% existing customers (14% attrited) meaning that a model that only guesses existing customers for every output will score higher than the demographic RFC. The RFC plot and AUC-ROC show similar results: the ROC is close to a straight line from the origin to (1, 1) meaning that there is an equal trade of between false positive rate and true positive rate when choosing a model threshold. The AUC-ROC score of 49% indicates that the ROC cuts the possibility space into two pieces, confirming that the ROC is approximately and straight line. 

All of this is to say that the demographics RFC model is not a great classifier and that there is probably not much correlation between demographics and attrition on their own. There may be other hidden correlations between demographics and usage features, but on their own they do not seem to train the model beyond random chance.


In [ ]:
Xdem = pd.read_csv('../Resources/X-dem.csv')

In [ ]:
features_dem = Xdem.columns.values
print(f'Feature names: {features_dem}')
print(f'Number of Features: {len(features_dem)}')

In [ ]:
print('Random forest classifiier on demographic data only')
print(f'Accuracy: {acct(Xdem, y, model="rfc")}')
print(f'Area under curve: {auct(Xdem, y, model="rfc", data_set="X-dem")}')

In [ ]:
plot_dem = pd.read_csv('../Outputs/X-dem_RandomForestClassifier_ROC.csv')
plt.plot(plot_dem['fpr'], plot_dem['tpr'])
plt.xlabel('False Postive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC for Random Forest Classifier Using Demographics')
plt.show()

In [ ]:
feature_importances_dem = fi(Xdem, y, model='rfc')
ticks = np.arange(len(features_dem))
fig, ax = plt.subplots()
ax.barh(ticks, feature_importances_dem)
ax.set_yticks(ticks, labels=features_dem)
fig.set_size_inches(15, 15)
ax.set_xlabel('Feature Importances')
ax.set_title('Feature Importances for Random Forest Classifier Using Demographics')
plt.show()
pd.DataFrame({'Features': features_dem, 'Feature_Importances': feature_importances_dem}).to_csv('../Outputs/X-dem_RandomForestClassifier_FI.csv')

## Evaluate Model with PCA
While the demographics data did not produce an RFC model that could make prediction better than chance, it is still possible to create an accurate model with less than the full set of data. PCA is used to project samples onto a subspace of the feature space that preserves the most variation to noise in the output. The PCA RFC model is trained with data that is dimensionally reduced using PCA and tested with data that uses the same PCA transformation. The testing accuracy score is 89% - similar to logistic regression and knn models. This model shows that it is possible to produce an accurate model with reduced dimensionality if the *right* combination of features are used.


In [ ]:
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.ensemble import RandomForestClassifier as RFC
from mechlearn import roc

_Xs, Xs_, _ypca, ypca_ = ss(X, y)
pca = PCA(n_components=21)
_Xspca, Xspca_ = pca.fit_transform(_Xs), pca.transform(Xs_)
print(f'Variance ratio of components: {[round(variance,2) for variance in pca.explained_variance_ratio_]}')
print(f'Number of components: 21')

In [ ]:
rfc_pca = RFC()
rfc_pca.fit(_Xspca, _ypca.to_numpy().ravel())
score = rfc_pca.score(Xspca_, ypca_)
auc = roc(Xspca_, ypca_, rfc_pca, 100, area=True, save_path='../Outputs/X-pca_RandomForestClassifier_ROC.csv')
print('Random forest classifiier with PCA')
print(f'Accuracy: {score}')
print(f'Area under curve: {auc}')

In [ ]:
plot_pca = pd.read_csv('../Outputs/X-pca_RandomForestClassifier_ROC.csv')
plt.plot(plot_pca['fpr'], plot_pca['tpr'])
plt.xlabel('False Postive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC for Random Forest Classifier with PCA')
plt.show()

## Conclusions
The base RFC model, trained with all features of the credit card churn data set, scores a high accuracy and appears to be a useful model for predicting credit card churn. In addition to the high accuracy of 95%, the model also possesses an ROC with a high 98% AUC meaning the model can be tuned to fit the use case. For both of these aspects, RFC outperforms the logistic regression and knn models and shows similar results to a neural network.  

Accuracy is lost when trying to make models trained on fewer features and the predictive power cannot fully match the model trained with all the features, which is expected. The demographics model does not show to perform any better than random chance and because of this, it does not make a good model for predicting credit card churn. The PCA model does perform better and on much larger sets of data this set might be necessary to save time and resources on model training. Still it is significantly less accurate than the base model and if the use case requires high accuracy, a neural net or the base model would be necessary.

Further analysis could explore using the demographics data to first predict key usage features and then use the usage features to predict churn (which has already been shown to be effective with the base model). This type of compounded model could be successful if a correlation exists between usage and demographics. The data set is constrained by only having around 16% attrited customers. Another thing to explore is synthesizing more attrited data to see if these models hold for a more balanced data set. 


## Other Analysis

In [ ]:
# this code generates a plot of to visualize how the prediction and targets is distributed across different slices of demographics
def feature_variance(df, feature, save_path=None, show_error=False, method='mean'):
    all_features = df.columns
    dfi = df.copy(deep=False)
    if not feature in all_features:
        type = 'cat'
        dfi[feature] = 1
        encoded_features = list(filter(lambda feature_: feature in feature_, all_features))
        for feature_ in encoded_features:
            dfi[feature] = dfi.apply(lambda x: feature_.replace(f'{feature}_', '') if x[feature_] == 1 else x[feature], axis=1)
    else:
        type = 'con'
    if method == 'mean':
        dfr = dfi.groupby(feature).mean()
        dfr['target_variance'] = dfi.groupby(feature).std()['target']
        dfr['prediction_variance'] = dfi.groupby(feature).std()['prediction']
    elif method == 'median':
        dfr.groupby(feature)
        dfr = dfi.groupby(feature).median()
    dfr.dropna(inplace=True)
    if type == 'cat':
        N = len(dfr)
        ind = np.arange(N)
        width = 0.3
        plt.bar(ind, dfr['prediction'], width, color='red', alpha=.5, label=f'{method.capitalize()} Prediction')
        plt.bar(ind + width, dfr['target'], width, color='blue', alpha=.5, label=f'{method.capitalize()} Target')
        plt.xticks(ind + width / 2, dfr.index, rotation=45)
        if show_error and method != 'median':
            plt.errorbar(ind, dfr['prediction'], dfr['prediction_variance'], color='red', fmt='o')
            plt.errorbar(ind + width, dfr['target'], dfr['target_variance'], color='blue', fmt='o')
    else:
        plt.plot(dfr.index, dfr['prediction'].values, c='red', label=f'{method.capitalize()} Prediction')
        plt.plot(dfr.index, dfr['target'], c='blue', label=f'{method.capitalize()} Target')
        if show_error and method != 'median':
            plt.fill_between(dfr.index, dfr['prediction']+dfr['prediction_variance'], y2=dfr['prediction']-dfr['prediction_variance'], color='red', alpha=.1)
            plt.fill_between(dfr.index, dfr['target']+dfr['target_variance'], y2=dfr['target']-dfr['target_variance'], color='blue', alpha=.1)
    plt.legend()
    plt.title(f'RFC Prediciton Across {feature.replace("_", " ").title()}')
    if type == 'con':
        plt.xlim(dfr.index.min(), dfr.index.max())
    plt.ylim(0 ,1)
    plt.xlabel(feature.replace("_", " ").title())
    plt.ylabel('Proportion')
    if not save_path == None:
        plt.savefig(save_path, bbox_inches='tight')
    plt.show()

In [ ]:
#  load data from model that was trained in model.py (and is used on website)
D_ = pd.read_csv('../resources/D_.csv')

# list all demographics
demographics = ['Customer_Age', 'Dependent_count', 'Gender', 'Education_Level', 'Marital_Status', 'Income_Category']

# create plot for each demographic
for demographic in demographics:
    feature_variance(D_, demographic, save_path=f'../static/assets/{demographic}_view.png', show_error=True)

In [ ]:
D_

In [ ]:
# plt.hist(rfc_dem_val['prediction'])
# plt.xlabel('Prediction Probability')
# plt.ylabel('Model Prediction Frequency')
# plt.title('Number of Times the Model Returns a\nPrediction Probability')
# plt.savefig('../Outputs/RFC-prediction-distribution.png')
# plt.show()